# Google Notebook specific Stuff

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pytorch-lightning==1.3.3
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl
!pip install scGeneFit
!pip install scanpy
!pip install anndata
!pip install lassonet
# my version of smashpy that fixed imports, nothing else was changed
!pip install matplotlib>=3.3.2
!pip install shap>=0.37.0
!pip install xgboost>=1.3.3
!pip install imbalanced-learn>=0.7.0
!pip install numba>=0.51.2
!pip install harmonypy>=0.0.5
!git clone https://github.com/beelze-b/SmashFork.git
!git clone https://github.com/ahsv/RankCorr.git

     |████████████████████████████████| 806 kB 4.0 MB/s 
     |████████████████████████████████| 282 kB 37.6 MB/s 
     |████████████████████████████████| 829 kB 31.9 MB/s 
     |████████████████████████████████| 125 kB 53.4 MB/s 
     |████████████████████████████████| 636 kB 57.3 MB/s 
     |████████████████████████████████| 1.3 MB 52.3 MB/s 
     |████████████████████████████████| 142 kB 55.6 MB/s 
     |████████████████████████████████| 294 kB 34.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=43fafcd8e394356327cf09990e055f9225b0f98d92efb5534bb8e286c63885bd
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [ ]:
# Check GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
import sys
import os
# to import utils
sys.path.append(os.path.abspath('/content/gdrive/My Drive/Differentiable-Sparse-Subset-Selection/notebooks'))

# All the Imports

In [ ]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from lassonet import LassoNetClassifier


from RankCorr.picturedRocks import Rocks
import SmashFork.smashpy as smashpy


import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part o

CPU times: user 23.7 s, sys: 1.18 s, total: 24.9 s
Wall time: 30.6 s


In [ ]:
from os import path

In [ ]:
model_save_path = '/content/gdrive/MyDrive/Differentiable-Sparse-Subset-Selection/data/{}/models/'
num_times = 5

  

## Accuracies and F1

In [ ]:
data_locations = [
                  'CITE-seq',
                  'developmental_mouse',
                  'mouse_brain_broad',
                  'paul',
                  'zeisel',
                  'synthetic'
]

In [ ]:
result_axis = [
 ('All Markers', 'all_markers_results_{}.npy', 'all_markers_knn_results_{}.npy'), 
 ('L1 VAE', 'l1_vae_results_{}.npy', 'l1_vae_knn_results_{}.npy'),
 ('Smash RF', 'smash_rf_results_{}.npy', 'smash_rf_knn_results_{}.npy'),
 ('SMaSH DNN', 'smash_results_{}.npy', 'smash_knn_results_{}.npy'),
 ('RankCorr', 'rankcorr_results_{}.npy', 'rankcorr_knn_results_{}.npy'),
 ('Global Gate VAE', 'globalgate_vae_results_{}.npy', 'globalgate_vae_knn_results_{}.npy'),
 ('RunningGumbel VAE', 'runningstate_vae_results_{}.npy', 'runningstate_vae_knn_results_{}.npy'),
 ('RunningGumbel Classifier', 'runningstate_classifier_results_{}.npy', 'runningstate_classifier_knn_results_{}.npy'),
 ('RunningGumbel VAE+Classifier', 'runningstate_vae+classifier_results_{}.npy', 'runningstate_vae+classifier_knn_results_{}.npy'),
 ('Concrete VAE',  'concrete_vae_results_{}.npy', 'concrete_vae_knn_results_{}.npy'),
 ('LassoNet', 'lasso_net_results_{}.npy', 'lasso_net_knn_results_{}.npy')
]

## Random Forest

In [ ]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,RunningGumbel VAE,RunningGumbel Classifier,RunningGumbel VAE+Classifier,Concrete VAE,LassoNet
CITE-seq,"(0.072, 0.907)","(0.123, 0.845)","(0.069, 0.915)","(0.067, 0.917)","(0.069, 0.912)","(0.126, 0.837)","(0.093, 0.882)","(0.063, 0.924)","(0.065, 0.921)","(0.124, 0.838)","(0.067, 0.919)"
developmental_mouse,"(0.135, 0.856)","(0.354, 0.622)","(0.138, 0.858)","(0.596, 0.392)","(0.135, 0.861)","(0.335, 0.661)","(0.281, 0.697)","(0.088, 0.908)","(0.246, 0.746)","(0.438, 0.533)","(0.092, 0.903)"
mouse_brain_broad,"(0.009, 0.991)","(0.148, 0.841)","(0.018, 0.982)","(0.014, 0.985)","(0.03, 0.969)","(0.09, 0.904)","(0.011, 0.989)","(0.005, 0.995)","(0.015, 0.985)","(0.163, 0.826)","(0.008, 0.992)"
paul,"(0.188, 0.792)","(0.534, 0.397)","(0.116, 0.881)","(0.192, 0.797)","(0.147, 0.847)","(0.362, 0.598)","(0.13, 0.868)","(0.139, 0.858)","(0.144, 0.852)","(0.412, 0.541)","(0.122, 0.873)"
zeisel,"(0.046, 0.952)","(0.235, 0.743)","(0.049, 0.95)","(0.08, 0.918)","(0.053, 0.946)","(0.203, 0.783)","(0.155, 0.835)","(0.049, 0.95)","(0.056, 0.943)","(0.198, 0.791)","(0.049, 0.95)"
synthetic,"(0.098, 0.901)","(0.118, 0.882)","(0.106, 0.893)","(0.098, 0.901)","(0.109, 0.891)","(0.156, 0.841)","(0.119, 0.881)","(0.088, 0.911)","(0.115, 0.885)","(0.146, 0.852)","(0.087, 0.912)"


## KNN

In [ ]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,RunningGumbel VAE,RunningGumbel Classifier,RunningGumbel VAE+Classifier,Concrete VAE,LassoNet
CITE-seq,"(0.093, 0.904)","(0.178, 0.798)","(0.073, 0.915)","(0.067, 0.921)","(0.081, 0.905)","(0.2, 0.77)","(0.116, 0.874)","(0.062, 0.929)","(0.069, 0.921)","(0.182, 0.789)","(0.064, 0.928)"
developmental_mouse,"(0.462, 0.48)","(0.519, 0.475)","(0.227, 0.775)","(0.696, 0.277)","(0.208, 0.782)","(0.477, 0.513)","(0.396, 0.589)","(0.115, 0.883)","(0.308, 0.688)","(0.577, 0.411)","(0.142, 0.845)"
mouse_brain_broad,"(0.099, 0.889)","(0.232, 0.742)","(0.028, 0.972)","(0.014, 0.985)","(0.04, 0.96)","(0.138, 0.851)","(0.014, 0.986)","(0.006, 0.994)","(0.02, 0.979)","(0.238, 0.74)","(0.009, 0.991)"
paul,"(0.331, 0.594)","(0.589, 0.37)","(0.212, 0.772)","(0.294, 0.677)","(0.249, 0.739)","(0.416, 0.553)","(0.244, 0.747)","(0.226, 0.766)","(0.274, 0.712)","(0.477, 0.488)","(0.226, 0.762)"
zeisel,"(0.05, 0.95)","(0.308, 0.677)","(0.055, 0.945)","(0.094, 0.904)","(0.056, 0.943)","(0.275, 0.713)","(0.185, 0.806)","(0.04, 0.96)","(0.067, 0.931)","(0.272, 0.711)","(0.061, 0.935)"
synthetic,"(0.216, 0.767)","(0.223, 0.763)","(0.177, 0.813)","(0.159, 0.832)","(0.179, 0.808)","(0.342, 0.64)","(0.203, 0.785)","(0.143, 0.851)","(0.221, 0.764)","(0.335, 0.649)","(0.139, 0.856)"


## Difference between results

In [ ]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m1, f11, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m2, f12, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m1-m2, 3)
    f1 = round(f11-f12, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results3 = pd.DataFrame(results)
tabulated_results3

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,RunningGumbel VAE,RunningGumbel Classifier,RunningGumbel VAE+Classifier,Concrete VAE,LassoNet
CITE-seq,"(-0.021, 0.003)","(-0.055, 0.047)","(-0.004, 0.001)","(0.0, -0.003)","(-0.012, 0.007)","(-0.074, 0.067)","(-0.023, 0.008)","(0.001, -0.005)","(-0.004, 0.0)","(-0.058, 0.049)","(0.003, -0.009)"
developmental_mouse,"(-0.327, 0.376)","(-0.165, 0.147)","(-0.088, 0.083)","(-0.1, 0.115)","(-0.073, 0.079)","(-0.142, 0.148)","(-0.115, 0.108)","(-0.027, 0.025)","(-0.062, 0.058)","(-0.138, 0.121)","(-0.05, 0.058)"
mouse_brain_broad,"(-0.09, 0.102)","(-0.084, 0.1)","(-0.01, 0.01)","(0.0, -0.0)","(-0.01, 0.01)","(-0.049, 0.053)","(-0.003, 0.003)","(-0.0, 0.0)","(-0.005, 0.005)","(-0.075, 0.087)","(-0.001, 0.001)"
paul,"(-0.143, 0.198)","(-0.054, 0.027)","(-0.096, 0.109)","(-0.102, 0.12)","(-0.101, 0.108)","(-0.055, 0.045)","(-0.114, 0.121)","(-0.087, 0.093)","(-0.13, 0.139)","(-0.066, 0.053)","(-0.104, 0.111)"
zeisel,"(-0.003, 0.002)","(-0.073, 0.066)","(-0.006, 0.006)","(-0.013, 0.014)","(-0.002, 0.003)","(-0.072, 0.07)","(-0.03, 0.029)","(0.009, -0.01)","(-0.012, 0.012)","(-0.074, 0.08)","(-0.012, 0.015)"
synthetic,"(-0.117, 0.134)","(-0.105, 0.119)","(-0.071, 0.081)","(-0.061, 0.069)","(-0.07, 0.083)","(-0.186, 0.201)","(-0.084, 0.096)","(-0.055, 0.06)","(-0.106, 0.121)","(-0.189, 0.203)","(-0.051, 0.057)"


# Things to note for paper

PySmash uses a seed internally so the model is deterministic to a particular dataset. We used various different train seeds, whereas the smash paper uses the whole dataset for the data.